In [1]:
import random
import math
import numpy as np
import numba as nb
from numba import njit

In [11]:
@njit
def funcion_p(a, h):
    valor1 = (2 * a + 1)
    valor2 = (a + 1) ** h
    valor3 = 0
    for k in range(0, a + 1):
        valor3 += k ** h
    valor3 *= 2
    return valor1 * valor2 - valor3

@njit
def prob_triangular_discreta(a, h, c, d):
    if d < c - a or d > c + a:
        return 0
    numerador = (a + 1) ** h - abs(d - c) ** h
    denominador = funcion_p(a, h)
    prob = numerador / denominador
    return prob

@njit
def va_triangular_discreta(a, h, c):
    u = np.random.random()
    # print(u)
    p_acum = 0
    for d in range(c - a, c + a + 1):
        prob = prob_triangular_discreta(a, h, c, d)
        if p_acum <= u < p_acum + prob:
            return d
        p_acum += prob
    return d

@njit
def esperanza_triangular_discreta(a, h, c):
    esperanza = 0
    for d in range(c - a, c + a + 1):
        esperanza += d * prob_triangular_discreta(a, h, c, d)
    return round(esperanza)

In [3]:
@njit
def a_value(t):
    if t <= 15:
        return 50
    elif t <= 30:
        return 86
    elif t <= 45:
        return 34
    elif t <= 60:
        return 22
    elif t <= 75:
        return 100
    return 66

@njit
def h_value(t):
    if t <= 15:
        return 5
    elif t <= 30:
        return 4
    elif t <= 45:
        return 4
    elif t <= 60:
        return 5
    elif t <= 75:
        return 3
    return 4

@njit
def c_value(t):
    if t <= 15:
        return 50
    elif t <= 30:
        return 86
    elif t <= 45:
        return 34
    elif t <= 60:
        return 22
    elif t <= 75:
        return 100
    return 66

@njit
def K_value(t):
    if t <= 30:
        return 180
    elif t <= 60:
        return 500
    return 250

@njit
def C_value(t):
    if (t % 7 == 0):
        return 18
    return 12

@njit
def H_value(t):
    return 1

@njit
def Q_value(t):
    return 18

In [22]:
@njit
def esperanza_triangular_discreta(a, h, c):
    esperanza = 0
    for d in range(c - a, c + a + 1):
        esperanza += d * prob_triangular_discreta(a, h, c, d)
    return round(esperanza)

29

In [4]:
@njit
def politica_optima(T, Q):
    array_size = (T+1, Q + 1)
    matriz_costos = np.zeros(array_size)
    matriz_decision = np.zeros(array_size)
    matriz_costos_inmediatos = np.zeros((Q + 1, Q + 1))

    for s in range(0, Q + 1):
        matriz_costos[T][s] = 0

    for t in range(T - 1, -1, -1):
        for s in range(0, Q + 1):
            #escoge acción óptima.
            minimum = np.inf
            x_value = -1
            for x in range(0, Q - s + 1):
                #costo inmediato
                CostoAccion = 0
                if x > 0:
                    CostoAccion += K_value(t+1) + C_value(t+1) * x
                for d in range(c_value(t+1) - a_value(t+1), c_value(t+1) + a_value(t+1) + 1):
                    CostoAccion += Q_value(t+1) * max(0, d - s - x) * prob_triangular_discreta(a_value(t+1), h_value(t+1), c_value(t+1), d)
                    CostoAccion += H_value(t+1) * max(0, s + x - d) * prob_triangular_discreta(a_value(t+1), h_value(t+1), c_value(t+1), d)
                #valor esperado futuro
                for d in range(c_value(t+1) - a_value(t+1), c_value(t+1) + a_value(t+1) + 1):
                    sfuturo = max(s + x - d, 0)
                    CostoAccion += matriz_costos[t+1][sfuturo] * prob_triangular_discreta(a_value(t+1), h_value(t+1), c_value(t+1), d)

                #Actualiza
                if CostoAccion< minimum:
                    minimum=CostoAccion
                    x_value=x

            matriz_costos[t][s] = minimum
            matriz_decision[t][s] = x_value


    return matriz_costos, matriz_decision

In [15]:
T = 90
Q = 120

matriz_costos, matriz_decision = politica_optima(T, Q)

In [17]:
for i in range(T+1):
    print(i+1, matriz_costos[i])
    # print(i+1, matriz_decision[i])

1 [83559.29556453 83547.29556453 83535.29556453 83523.29556453
 83511.29556453 83499.29556453 83487.29556453 83475.29556453
 83463.29556453 83451.29556453 83439.29556453 83427.29556453
 83415.29556453 83403.29556453 83391.29556453 83379.29556453
 83367.29556453 83355.29556453 83343.29556453 83331.29556453
 83319.29556453 83307.29556453 83295.29556453 83283.29556453
 83271.29556453 83259.29556453 83247.29556453 83235.29556453
 83223.29556453 83211.29556453 83199.29556453 83183.47239716
 83167.44012217 83151.47874225 83135.58487993 83119.75548388
 83103.98780765 83088.27938937 83072.62803263 83057.0317882
 83041.48893676 83025.99797242 83010.55758718 82995.16665611
 82979.82422338 82964.52948898 82949.28179618 82934.08061975
 82918.92555477 82903.81630618 82888.75267899 82873.73456911
 82858.76195481 82843.83488883 82828.95349107 82814.11794174
 82799.32847501 82784.58537303 82769.88896032 82755.23959835
 82740.6376805  82726.08362707 82711.57788043 82697.12090028
 82682.71315894 82668.3

In [7]:
print(matriz_costos[0][40])

83041.48893676212


In [8]:
@njit
def simular_demanda(cantidad_iteraciones, T):
    out = ""
    for i in range(cantidad_iteraciones):
        for t in range(1, T + 1):
            a = a_value(t)
            h = h_value(t)
            c = c_value(t)
            demanda = va_triangular_discreta(a, h, c)
            out += str(demanda) + ","
        out = out[:-1]
        out += "\n"
    return out

In [18]:
def simulacion_aplicando_politica_optima(demanda, matriz_decision):
    stock_inicial = 40
    stock = stock_inicial
    costo_total = 0
    for t in range(1, T + 1):
        # print("Tiempo:", t)
        # print("Stock:", stock)
        # print("Demanda:", demanda[t - 1])
        # print("Decision:", matriz_decision[t - 1][int(stock)])
        reposicion = matriz_decision[t - 1][stock]
        print(f"En {t} se ordenan {reposicion} unidades")
        # print("Reposicion:", reposicion)
        if reposicion > 0:
            costo_total += K_value(t) + C_value(t) * reposicion
        costo_total += Q_value(t) * max(0, demanda[t - 1] - stock - reposicion)
        # print(Q_value(t) * max(0, demanda[t - 1] - stock - reposicion))
        costo_total += H_value(t) * max(0, stock + reposicion - demanda[t - 1])
        # print(H_value(t) * max(0, stock + reposicion - demanda[t - 1]))
        # print("Stock - Demanda:", stock)
        stock = int(max(stock + reposicion - demanda[t - 1], 0))
        # print("Costo:", costo_total)
        # print()
    return costo_total

In [19]:
M = 500
output = simular_demanda(M, T)

with open("demanda.csv", "w") as f:
    f.write(output)

with open("demanda.csv", "r") as f:
    demanda = f.readlines()

suma = 0
for i in range(M):
    dem = demanda[i].split(",")
    for j in range(len(dem)):
        dem[j] = int(dem[j])
    suma += simulacion_aplicando_politica_optima(dem, matriz_decision)

print(suma / M)

En 1 se ordenan 0.0 unidades
En 2 se ordenan 120.0 unidades
En 3 se ordenan 0.0 unidades
En 4 se ordenan 0.0 unidades
En 5 se ordenan 106.0 unidades
En 6 se ordenan 65.0 unidades
En 7 se ordenan 0.0 unidades
En 8 se ordenan 110.0 unidades
En 9 se ordenan 0.0 unidades
En 10 se ordenan 0.0 unidades
En 11 se ordenan 118.0 unidades
En 12 se ordenan 0.0 unidades
En 13 se ordenan 92.0 unidades
En 14 se ordenan 0.0 unidades
En 15 se ordenan 0.0 unidades
En 16 se ordenan 120.0 unidades
En 17 se ordenan 118.0 unidades
En 18 se ordenan 70.0 unidades
En 19 se ordenan 109.0 unidades
En 20 se ordenan 110.0 unidades
En 21 se ordenan 0.0 unidades
En 22 se ordenan 120.0 unidades
En 23 se ordenan 118.0 unidades
En 24 se ordenan 120.0 unidades
En 25 se ordenan 66.0 unidades
En 26 se ordenan 0.0 unidades
En 27 se ordenan 120.0 unidades
En 28 se ordenan 0.0 unidades
En 29 se ordenan 120.0 unidades
En 30 se ordenan 42.0 unidades
En 31 se ordenan 0.0 unidades
En 32 se ordenan 0.0 unidades
En 33 se ordenan 0